In [1]:
import os
os.chdir("..")
os.getcwd()

'd:\\Projects\\Black Hippo\\black-hippo-experiments'

In [36]:
import pprint
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
from src.ai.vector_db_connector import VectorDBConnector
from src.common.schema import QdrantRecord
from src.ai.describe_image import AnswerSchema, describe_image
from src.services.blob_service import BlobService

In [3]:
db_conn = VectorDBConnector()

In [ ]:
db_conn.create_colection()

In [5]:
db_conn.list_collections()

CollectionsResponse(collections=[CollectionDescription(name='reference_data')])

In [34]:
db_conn.recreate_colection()

In [9]:
train_df = pd.read_csv("resources/train_df.csv")
embedder = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")
bs = BlobService()

In [16]:
def row_to_text_input(df: pd.DataFrame, i: int) -> str:
    text = f"""
    Supplier name = {df["supplier_name"].iloc[i]}
    Product name = {df["supplier_reference_description"].iloc[i]}
    Product price = {df["purchase_price"].iloc[i]}
    Materials = {df["materials"].iloc[i]}
    Possible weight (grams): = {df["weight"].iloc[i]}
    Lenght (cm) = {df["length"].iloc[i]}
    Height (cm) = {df["height"].iloc[i]}
    Width (cm) = {df["width"].iloc[i]}
    """
    return text

In [ ]:
records = []

for i in tqdm(range(len(train_df))):
    row = train_df.iloc[i]
    text = row_to_text_input(train_df, i)
    records.append(dict(
        embedding=embedder.encode(text),
        metadata={
            x: row[x]
            for x in [
                "item_id",
                "supplier_name",
                "supplier_reference_description",
                "purchase_price",
                "materials",
                "weight",
                "length",
                "height",
                "width",
                "main",
                "sub",
                "detail",
                "level4"
            ]
        }
    ))

records = [QdrantRecord(**x) for x in records]

100%|██████████| 6823/6823 [11:37<00:00,  9.78it/s]


In [35]:
start, stop = 0, 100
while True:
    if len(records) < stop:
        stop = len(records)
        db_conn.upsert_records(records[start:stop])
        break
    elif len(records) == stop:
        db_conn.upsert_records(records[start:stop])
        break
    db_conn.upsert_records(records[start:stop])
    start = stop
    stop += 100
    

In [37]:
sim_records = db_conn.similarity_search(
    embedding=embedder.encode(row_to_text_input(train_df, 0)),
    limit=5
)
pprint.pp(sim_records)

[ScoredPoint(id='b3f6fef7-c5dc-423c-bc9a-e79b7dac647e', version=10, score=1.0, payload={'item_id': 73205.0, 'supplier_name': 'Umiss ', 'supplier_reference_description': '120gsm paper with magnet ， no metallic gold paint   Size：18.5*H25cm', 'purchase_price': 1.15, 'materials': 'PAPER (95.00%), MAGNETS (5.00%)', 'weight': 60.0, 'length': 18.5, 'height': 25.0, 'width': 18.5, 'main': 'Decoration', 'sub': 'Home Accessories', 'detail': 'Decorative Objects', 'level4': 'Decorative Trees'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id='3b733926-1009-401f-bf6e-3009e292ff88', version=16, score=1.0, payload={'item_id': 73199.0, 'supplier_name': 'Umiss ', 'supplier_reference_description': '120gsm paper with magnet ， no metallic gold paint   Size：18.5*H25cm', 'purchase_price': 1.15, 'materials': 'PAPER (95.00%), MAGNETS (5.00%)', 'weight': 60.0, 'length': 18.5, 'height': 25.0, 'width': 18.5, 'main': 'Decoration', 'sub': 'Home Accessories', 'detail': 'Decorative Objects', 'level4':

In [38]:
pprint.pp([x.model_dump() for x in sim_records])

[{'id': 'b3f6fef7-c5dc-423c-bc9a-e79b7dac647e',
  'version': 10,
  'score': 1.0,
  'payload': {'item_id': 73205.0,
              'supplier_name': 'Umiss ',
              'supplier_reference_description': '120gsm paper with magnet ， no '
                                                'metallic gold paint   '
                                                'Size：18.5*H25cm',
              'purchase_price': 1.15,
              'materials': 'PAPER (95.00%), MAGNETS (5.00%)',
              'weight': 60.0,
              'length': 18.5,
              'height': 25.0,
              'width': 18.5,
              'main': 'Decoration',
              'sub': 'Home Accessories',
              'detail': 'Decorative Objects',
              'level4': 'Decorative Trees'},
  'vector': None,
  'shard_key': None,
  'order_value': None},
 {'id': '3b733926-1009-401f-bf6e-3009e292ff88',
  'version': 16,
  'score': 1.0,
  'payload': {'item_id': 73199.0,
              'supplier_name': 'Umiss ',
              'su